# GEOQUERIES

In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import json
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from bson.json_util import dumps
import numpy as np

In [2]:
#london_loc = {'type': 'Point', 'coordinates':[ -0.0870022262169723, 51.520624794291884]}
#singapore_loc = {'type': 'Point', 'coordinates':[103.84868192263733, 1.2862863534171092]}
#san_francisco_loc = {'type': 'Point', 'coordinates':[ -122.41049370727525,37.79787988216447]}
london_loc1 = [-0.0870022262169723, 51.520624794291884]
singapore_loc1 = [103.84868192263733, 1.2862863534171092]
san_francisco_loc1 =[ -122.41049370727525,37.79787988216447]

In [3]:
metros = 2500

In [4]:
#Crear la conexión con el database de Londres dentro de MongoDB
client = MongoClient("localhost:27017")
db = client.get_database("Ciudades")

In [5]:
ln = db.get_collection("London")
sg = db.get_collection("Singapore")
sf = db.get_collection("San Francisco")


# FUNCIONES

In [6]:
'''
Esta función recibe la coleccion con la que quieres trabajar, la localizacion establecida, el maximo de distancia
que quieres y la classificación que quieres buscar.
Te devuelve sitios cercanos a la ubicación ajustandose a las condiciones(maxima distancia, por classificacion y por
localización)

'''
def coger_sitios_cercanos(collection, location, Classification, max_distance):
    var_schools = {"$and":[{"location2": {"$near": {"$geometry": location,"$minDistance":0,"$maxDistance": max_distance}}},{"Classification": f'{Classification}'}]}
    encontrar_colegios = collection.find(var_schools)
    return list(encontrar_colegios)
    

In [7]:
#Tipos de classificación/Type
#1. School: Type
#2. Technological Startups
#3. Designing Studios
#4. Leisure Places
#5. Travelling Stations
#6. Parks

In [8]:
# A esta función entra la collection de mongoDB con la que queremos trabajar y la localización establecida.
# Esta función nos crea un dataframe nuevo con la distacia de cada sitio a la localización establecida. 
def get_distances(collection, location1):
    query = [{'$geoNear':{'near':location1,
    'distanceField' : "distance",
    'maxDistance': 1,
    'distanceMultiplier': 6371*1000 ,
    'spherical': True}}]
    geoloc = collection.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    df = pd.DataFrame(response_json)
    df1 = df[['Name', 'Classification', 'Type', 'Country', 'Adress', 'Latitude', 'Longitude', 'location2', 'distance']]
    return df1
    


In [9]:
# En estas 3 variables, tenemos el dataframe con la distacia a nuestro punto inicial en metros.
london_distances = get_distances(ln, london_loc1)
san_francisco_distances = get_distances(sf, san_francisco_loc1)
singapore_distances = get_distances(sg, singapore_loc1)

In [10]:
london_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198


In [11]:
san_francisco_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Jean Parker Elementary,School,Elementary School,US,840 Broadway,37.797641,-122.411043,"{'type': 'Point', 'coordinates': [-122.411043,...",55.089767
1,Broadway Tunnel East Mini Park,Parks,Park,US,967 Broadway (at Himmelman),37.797354,-122.410502,"{'type': 'Point', 'coordinates': [-122.4105024...",58.444345


In [12]:
singapore_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Liquid 40,Leisure Places,Pub,SG,Circular Rd,1.286381,103.848842,"{'type': 'Point', 'coordinates': [103.84884158...",20.615074
1,Audition pub,Leisure Places,Pub,SG,76 Circular Road,1.286644,103.848966,"{'type': 'Point', 'coordinates': [103.84896614...",50.772528


In [13]:
london_distances

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198
2,The Flying Horse,Leisure Places,Pub,GB,52 Wilson St (Sun St),51.520375,-0.085176,"{'type': 'Point', 'coordinates': [-0.085176182...",129.349477
3,Singer Tavern,Leisure Places,Bar,GB,1 City Rd,51.522259,-0.086933,"{'type': 'Point', 'coordinates': [-0.086932681...",181.762518
4,Red Lion,Leisure Places,Pub,GB,1 Eldon St (South Pl),51.518843,-0.086181,"{'type': 'Point', 'coordinates': [-0.086181312...",206.137330
...,...,...,...,...,...,...,...,...,...
350,Bright Horizons Mount Carmel Day Nursery and P...,School,Nursery School,GB,18 Eden Grove,51.551394,-0.112786,"{'type': 'Point', 'coordinates': [-0.112786369...",3858.263739
351,Monkey Puzzle Day Nursery,School,Nursery School,GB,"Loxfords House, Highbury Park, N5",51.556389,-0.098601,"{'type': 'Point', 'coordinates': [-0.098600901...",4056.887769
352,City Academy Hackney,School,High School,GB,Homerton Row,51.548704,-0.048845,"{'type': 'Point', 'coordinates': [-0.048845102...",4088.335992
353,Camden School for Girls,School,High School,GB,Sandall Street,51.547697,-0.134395,"{'type': 'Point', 'coordinates': [-0.134395257...",4450.610518


# PONDERACION

In [14]:
# Para escoger la localización, debemos dar importancia a las condiciones escogidas

In [15]:
def eliminar_a_mas_de_1km (df):
    df = df.drop(df[df.distance > 1000].index)
    return df

In [16]:
lnd_final_definitivo = eliminar_a_mas_de_1km(london_distances)

In [17]:
sgp_final_definitivo = eliminar_a_mas_de_1km(singapore_distances)

In [18]:
sf_final_definitivo = eliminar_a_mas_de_1km(san_francisco_distances)

In [19]:
parks_pond = 0.05
school_pond = 0.30
technological_Startups_pond = 0.05
designing_Studios_pond = 0.25
travelling_Stations_pond = 0.2
leisure_pond = 0.15

In [20]:
len(lnd_final_definitivo.Classification.unique())

6

In [21]:
sgp_final_definitivo.Classification.unique()
#Como vemos Singapur no tiene colegios a menos de 1 km, por lo tanto podemos ir descartandolo.

array(['Leisure Places', 'Designing Studios', 'Technological Startups',
       'Parks', 'Travelling Stations'], dtype=object)

In [22]:
len(sf_final_definitivo.Classification.unique())

6

In [23]:
#def añadir_ponderaciones(df):
    df['valor'] = np.where(df['Classification'] == 'Parks',df['distance'] * parks_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Leisure Places',df['distance'] * leisure_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Technological Startups',df['distance'] * technological_Startups_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Designing Studios',df['distance'] * designing_Studios_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Travelling Stations',df['distance'] * travelling_Stations_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'School',df['distance'] * school_pond,df['distance'])
    return df

IndentationError: unexpected indent (685976021.py, line 2)

In [24]:
lnd_final_definitivo


,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198
2,The Flying Horse,Leisure Places,Pub,GB,52 Wilson St (Sun St),51.520375,-0.085176,"{'type': 'Point', 'coordinates': [-0.085176182...",129.349477
3,Singer Tavern,Leisure Places,Bar,GB,1 City Rd,51.522259,-0.086933,"{'type': 'Point', 'coordinates': [-0.086932681...",181.762518
4,Red Lion,Leisure Places,Pub,GB,1 Eldon St (South Pl),51.518843,-0.086181,"{'type': 'Point', 'coordinates': [-0.086181312...",206.137330
...,...,...,...,...,...,...,...,...,...
180,BSG (UK),Technological Startups,Tech Startup,GB,230 City Road,51.528608,-0.093074,"{'type': 'Point', 'coordinates': [-0.093074241...",982.054817
181,The Vintry,Leisure Places,Pub,GB,"Abchurch Yard, Off Abchurch Lane (off Cannon St)",51.511854,-0.088678,"{'type': 'Point', 'coordinates': [-0.088678, 5...",982.103957
182,LIVEPOSTER HQ,Technological Startups,Tech Startup,GB,19-20 Gt Sutton St.,51.523271,-0.100551,"{'type': 'Point', 'coordinates': [-0.100550651...",982.458814
183,The George & Vulture,Leisure Places,Pub,GB,63 Pitfield St (at Haberdasher St),51.529294,-0.083685,"{'type': 'Point', 'coordinates': [-0.083684969...",990.894513


In [41]:
def conditions(df):
    for i in df['Classification']:
        if (i == 'Parks'):
            return (df['distance'] * parks_pond)
        elif (i == 'Leisure Places'):
            return (df['distance'] * leisure_pond)
        elif (i == 'Technological Startups'):
            return (df['distance'] * technological_Startups_pond)
        elif (i == 'Designing Studios'):
            return (df['distance'] * designing_Studios_pond)
        elif (i == 'Travelling Stations'):
            return (df['distance'] * travelling_Stations_pond)
        elif (i == 'School'):
            return (df['distance'] * school_pond)
        
        
        
    

In [43]:
lnd_final_definitivo["valor"] = conditions(lnd_final_definitivo)

In [46]:
lnd_final_definitivo.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance,valor
62,Alchemist Logo Design,Designing Studios,Design Studio,GB,86-90 Paul Street,51.525401,-0.083738,"{'type': 'Point', 'coordinates': [-0.083738194...",577.123691,28.856185
161,The Britannia,Leisure Places,Pub,GB,"94 Ironmonger Row, Clerkenwell",51.527303,-0.094463,"{'type': 'Point', 'coordinates': [-0.094462896...",904.308101,45.215405
179,The Paternoster,Leisure Places,Pub,GB,Paternoster Sq (2-4 Queens Head Passage),51.515309,-0.098318,"{'type': 'Point', 'coordinates': [-0.098318391...",981.071186,49.053559
86,Village Underground,Leisure Places,Nightclub,GB,54 Holywell Ln,51.523552,-0.078558,"{'type': 'Point', 'coordinates': [-0.078558157...",668.784104,33.439205
105,Prince Arthur,Leisure Places,Pub,GB,49 Brunswick Pl,51.527337,-0.085227,"{'type': 'Point', 'coordinates': [-0.085227122...",756.371146,37.818557
100,the WOW wall,Designing Studios,Design Studio,GB,152 City Road,51.527236,-0.088792,"{'type': 'Point', 'coordinates': [-0.088791847...",745.441234,37.272062
162,Royal Oak Court,Parks,Park,GB,United Kingdom,51.528508,-0.083596,"{'type': 'Point', 'coordinates': [-0.083595815...",907.690545,45.384527
115,Strongroom Bar,Leisure Places,Pub,GB,120-124 Curtain Rd,51.526046,-0.079714,"{'type': 'Point', 'coordinates': [-0.079713715...",785.945537,39.297277
159,Religion Clothing,Designing Studios,Office,GB,"5 Standard Place, Rivington Street",51.526529,-0.078046,"{'type': 'Point', 'coordinates': [-0.078045958...",902.718348,45.135917
13,Geckoboard HQ,Technological Startups,Tech Startup,GB,"3rd Floor, 60 Worship Street",51.521949,-0.082773,"{'type': 'Point', 'coordinates': [-0.082772970...",327.562744,16.378137


In [35]:
def añadir_ponderaciones(df):
    df['valor'] = np.where(df['Classification'] == 'Parks',df['distance'] * parks_pond,df['distance'])

    return df

In [36]:
añadir_ponderaciones(lnd_final_definitivo)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance,valor
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274,1.738814
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198,120.590198
2,The Flying Horse,Leisure Places,Pub,GB,52 Wilson St (Sun St),51.520375,-0.085176,"{'type': 'Point', 'coordinates': [-0.085176182...",129.349477,129.349477
3,Singer Tavern,Leisure Places,Bar,GB,1 City Rd,51.522259,-0.086933,"{'type': 'Point', 'coordinates': [-0.086932681...",181.762518,181.762518
4,Red Lion,Leisure Places,Pub,GB,1 Eldon St (South Pl),51.518843,-0.086181,"{'type': 'Point', 'coordinates': [-0.086181312...",206.137330,206.137330
...,...,...,...,...,...,...,...,...,...,...
180,BSG (UK),Technological Startups,Tech Startup,GB,230 City Road,51.528608,-0.093074,"{'type': 'Point', 'coordinates': [-0.093074241...",982.054817,982.054817
181,The Vintry,Leisure Places,Pub,GB,"Abchurch Yard, Off Abchurch Lane (off Cannon St)",51.511854,-0.088678,"{'type': 'Point', 'coordinates': [-0.088678, 5...",982.103957,982.103957
182,LIVEPOSTER HQ,Technological Startups,Tech Startup,GB,19-20 Gt Sutton St.,51.523271,-0.100551,"{'type': 'Point', 'coordinates': [-0.100550651...",982.458814,982.458814
183,The George & Vulture,Leisure Places,Pub,GB,63 Pitfield St (at Haberdasher St),51.529294,-0.083685,"{'type': 'Point', 'coordinates': [-0.083684969...",990.894513,990.894513


In [37]:
def añadir_ponderaciones(df):
    df['valor'] = np.where(df['Classification'] == 'Parks',df['distance'] * parks_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Leisure Places',df['distance'] * leisure_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Technological Startups',df['distance'] * technological_Startups_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Designing Studios',df['distance'] * designing_Studios_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'Travelling Stations',df['distance'] * travelling_Stations_pond,df['distance'])
    df['valor'] = np.where(df['Classification'] == 'School',df['distance'] * school_pond,df['distance'])
    return df